In [1]:

!pip install darts

Defaulting to user installation because normal site-packages is not writeable
  Using cached packaging-24.2-py3-none-any.whl (65 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bokeh 3.7.0 requires contourpy>=1.2, but you have contourpy 1.1.0 which is incompatible.
dask 2025.3.0 requires cloudpickle>=3.0.0, but you have cloudpickle 2.2.1 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
#os.environ["NUMEXPR_MAX_THREADS"] = "72"

from darts import TimeSeries
from darts.models import NBEATSModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mae, rmse
import os
import matplotlib.pyplot as plt
from darts import TimeSeries
from pytorch_lightning import Trainer
from darts.models import NBEATSModel
import torch
import numpy as np
import pandas as pd
from darts.models import RNNModel
%matplotlib inline

In [2]:
folder_path = "/gpfs/home5/talkanbar/"
file_name = "df_classified_good.pkl" # file for selected symbols (main experiments)
file_path = os.path.join(folder_path, file_name)
merged_df = pd.read_pickle(file_path)

In [3]:
num_rows = merged_df.shape[0]
print("Number of rows:", num_rows)
num_unique_symbols = merged_df['symbol'].nunique()
print("Number of unique symbols:", num_unique_symbols)

Number of rows: 9918123
Number of unique symbols: 369


### functions

In [3]:
#many functions were used in EDA and in debugging the training process including:
def inspect_short_symbols(df, short_symbols):
   # To inspects the minimum and maximum 'timestamp_fixed' values for the specified short symbols in the DataFrame
    for symbol in short_symbols:
        symbol_df = df[df['symbol'] == symbol].sort_values('timestamp_fixed')
        if not symbol_df.empty:
            min_ts = symbol_df['timestamp_fixed'].min()
            max_ts = symbol_df['timestamp_fixed'].max()
            print(f"Symbol: {symbol}")
            print(f"  Minimum timestamp_fixed: {min_ts}")
            print(f"  Maximum timestamp_fixed: {max_ts}")
            print(f"  Number of rows: {len(symbol_df)}")
        else:
            print(f"Symbol '{symbol}' not found in the DataFrame.")

def identify_short_time_series(df, H, covariate_cols, input_chunk_length):
    #Identifies symbols whose time series are too short for the given NBEATS model parameters after target shifting. 
    short_symbols = []
    min_required_length = max(input_chunk_length, 1 + H)
    grouped = df.groupby('symbol')
    for symbol, group_df in grouped:
        group_df = group_df.sort_values('timestamp_fixed').copy()
        group_df['target'] = group_df['mid_price_usd'].shift(-H)
        group_df_cleaned = group_df.dropna(subset=['target'] + covariate_cols)

        target_series = TimeSeries.from_dataframe(
            group_df_cleaned, time_col='timestamp_fixed', value_cols='target'
        )
        if len(target_series) < min_required_length:
            short_symbols.append(symbol)
    return short_symbols

### Preparing the file

filtering: each day:excluding last 2 hours

In [5]:
df=merged_df
#as explained in the paper, we excluded the last 2 hours
df = df[df['timestamp_fixed'].dt.time < (pd.Timestamp('06:00:00').time())]


deleting symbols that has less than 80% coverage

In [9]:
filtered_vol_df=df
symbol_counts = filtered_vol_df['symbol'].value_counts()
max_count = symbol_counts.max()
threshold = max_count * 0.8
symbols_to_keep = symbol_counts[symbol_counts >= threshold].index
df_filtered3 = filtered_vol_df[filtered_vol_df['symbol'].isin(symbols_to_keep)]
df = df_filtered3


In [43]:
identify_short_time_series(df, 100, covariate_cols, 500) #if none we are good to go

[]

# NBEATS

We can select one of these settings:

Raw

In [13]:
covariate_cols = [
    'asks_0__price_usd', 'asks_1__price_usd', 'asks_2__price_usd',
    'asks_3__price_usd', 'asks_4__price_usd', 'bids_0__price_usd',
    'bids_1__price_usd', 'bids_2__price_usd', 'bids_3__price_usd',
    'bids_4__price_usd', 'asks_0__amount_norm', 'asks_1__amount_norm',
    'asks_2__amount_norm', 'asks_3__amount_norm', 'asks_4__amount_norm',
    'bids_0__amount_norm', 'bids_1__amount_norm', 'bids_2__amount_norm',
    'bids_3__amount_norm', 'bids_4__amount_norm',
  
  
    'time_to_expiry_min'
    ,'strike'
    #, 'underlying_price'
    ,'type_C', 'type_P'
    , 'moneyness'
   # 'mid_price_volatility_norm', 'total_bid_depth', 'total_ask_depth', 'order_book_depth',
    #'relative_spread','imbalance_L1_pct','last_price', 'bid_iv', 'ask_iv', 'mark_price', 'weighted_mid_price_L1_usd_norm', 'weighted_mid_price_L2_usd_norm',
    #   'weighted_mid_price_L3_usd_norm', 'VWMP_usd_norm', 'spread_usd_norm'
    #,'delta', 'gamma',
    #   'vega', 'theta', 'rho', 'moneyness'
]
target_col = 'mid_price_usd'

with features

In [22]:
covariate_cols = [
    'asks_0__price_usd', 'asks_1__price_usd', 'asks_2__price_usd',
    'asks_3__price_usd', 'asks_4__price_usd', 'bids_0__price_usd',
    'bids_1__price_usd', 'bids_2__price_usd', 'bids_3__price_usd',
    'bids_4__price_usd', 'asks_0__amount_norm', 'asks_1__amount_norm',
    'asks_2__amount_norm', 'asks_3__amount_norm', 'asks_4__amount_norm',
    'bids_0__amount_norm', 'bids_1__amount_norm', 'bids_2__amount_norm',
    'bids_3__amount_norm', 'bids_4__amount_norm',
 
    'time_to_expiry_min'
    ,'strike'
 
    ,'type_C', 'type_P',
'open_interest', 'last_price', 'bid_iv',
       'ask_iv', 'mark_price', 'mark_iv',  
       'underlying_price', 'delta', 'gamma', 'vega', 'theta', 'rho', 
      'moneyness',  
       'underlying_return',   'imbalance_L1_usd',
       'imbalance_L1_pct', 'imbalance_L1_rolling_mean', 'imbalance_L2_usd',
       'imbalance_L3_usd', 'imbalance_VWMP_usd', 'spread_usd',
       'relative_spread', 'depth_ratio', 'price_impact_ratio',
       'total_bid_depth', 'total_ask_depth', 'order_book_depth'
]
target_col = 'mid_price_usd'

In [ ]:

#now training and evaluataion pipeline
#  prediction horizon
H = 100  

# the  following settings are constructed so we train onoe model for all symbols
clean_groups = {}
for symbol, group_df in df.groupby('symbol'):
    group_df = group_df.sort_values('timestamp_fixed').copy()
    # Create the shifted target
    group_df['target'] = group_df['mid_price_usd'].shift(-H)
    group_df = group_df.dropna(subset=['target'] + covariate_cols)
    float_cols = ['target', 'mid_price_usd'] + covariate_cols
    group_df[float_cols] = group_df[float_cols].astype(np.float32)
    clean_groups[symbol] = group_df.reset_index(drop=True)


raw_target_series = {}
raw_covariate_series = {}
for symbol, group_df in clean_groups.items():
    target_series = TimeSeries.from_dataframe(group_df, time_col='timestamp_fixed', value_cols='target' )
    covariate_series = TimeSeries.from_dataframe(group_df, time_col='timestamp_fixed', value_cols=covariate_cols)
    raw_target_series[symbol] = target_series
    raw_covariate_series[symbol] = covariate_series
# Parameters
train_ratio = 0.8

# Outputs
train_target_series = []
test_target_series = []
train_covariate_series = []
test_covariate_series = []
target_scalers = []
all_target_series_scaled = []
all_covariate_series_scaled = []
symbols = list(raw_target_series.keys())
for symbol in symbols:
    target_series = raw_target_series[symbol]
    covariate_series = raw_covariate_series[symbol]
    # Split 
    split_point = int(len(target_series) * train_ratio)

    # Fit scaler
    scaler_target = Scaler()
    scaler_covariate = Scaler()
    train_target = target_series[:split_point]
    train_covariate = covariate_series[:split_point]
    target_scaled = scaler_target.fit(train_target).transform(target_series)
    covariate_scaled = scaler_covariate.fit(train_covariate).transform(covariate_series)

    target_scalers.append(scaler_target)

    train_target_series.append(target_scaled[:split_point])
    test_target_series.append(target_scaled[split_point:])
    train_covariate_series.append(covariate_scaled[:split_point])
    test_covariate_series.append(covariate_scaled[split_point:])

    all_target_series_scaled.append(target_scaled)
    all_covariate_series_scaled.append(covariate_scaled)

#  the model
model = NBEATSModel(
    input_chunk_length=500,  
    output_chunk_length=H,
    generic_architecture=True,
    num_stacks=2,
    num_blocks=3,
    num_layers=4,
    layer_widths=512,
    n_epochs=1,  
    random_state=42
)

#We report metrics per symbol
from darts.metrics import mae, rmse, mape

model.fit(series=train_target_series, past_covariates=train_covariate_series, verbose=True)
mae_scores_per_symbol = {}
rmse_scores_per_symbol = {}
mape_scores_per_symbol = {}
predictions_per_symbol = {}

# Evaluate 
for i, symbol in enumerate(symbols):
    predicted = model.historical_forecasts(
        series=all_target_series_scaled[i],
        past_covariates=all_covariate_series_scaled[i],
        start=len(train_target_series[i]),
        forecast_horizon=H,
        stride=1,
        retrain=False,
        verbose=False  
    )
    predictions_per_symbol[symbol] = predicted

#  per symbol
for i, symbol in enumerate(symbols):
    actual = target_scalers[i].inverse_transform(test_target_series[i])
    predicted = target_scalers[i].inverse_transform(predictions_per_symbol[symbol])

    mae_val = mae(actual, predicted)
    rmse_val = rmse(actual, predicted)
    try:
        mape_val = mape(actual, predicted)
    except ValueError as e:
        print(f"MAPE error for symbol {symbol}: {e}")
        mape_val = np.nan

    mae_scores_per_symbol[symbol] = mae_val
    rmse_scores_per_symbol[symbol] = rmse_val
    mape_scores_per_symbol[symbol] = mape_val

#  average metrics
avg_mae = np.nanmean(list(mae_scores_per_symbol.values()))
avg_rmse = np.nanmean(list(rmse_scores_per_symbol.values()))
avg_mape = np.nanmean(list(mape_scores_per_symbol.values()))

metrics_df = pd.DataFrame({
    'Symbol': list(mae_scores_per_symbol.keys()),
    'MAE': list(mae_scores_per_symbol.values()),
    'RMSE': list(rmse_scores_per_symbol.values()),
    'MAPE': list(mape_scores_per_symbol.values())
})


print(metrics_df)

print(f"Average MAE with historical_forecasts: {avg_mae:.4f}")
print(f"Average RMSE with historical_forecasts: {avg_rmse:.4f}")
print(f"Average MAPE with historical_forecasts: {avg_mape:.4f}")

metrics_df.to_pickle('metrics_df_one4all_goodsymbols_allfeatures_NB.pkl')
#we did this experiment for many settings of the data (from initial experiments to main experiments) and each time we saved the metrics df

###  LSTM RNN

In [ ]:
#We use same settings as NBEATS, except what is specific to RNN
H = 100  # Prediction horizon
clean_groups = {}

for symbol, group_df in df.groupby('symbol'):
    group_df = group_df.sort_values('timestamp_fixed').copy()
    group_df['target'] = group_df['mid_price_usd'].shift(-H)
    numeric_cols = [col for col in covariate_cols if col != 'timestamp_fixed']
    group_df[numeric_cols + ['target', 'mid_price_usd']] = group_df[numeric_cols + ['target', 'mid_price_usd']].astype(np.float32)
    if 'time_to_expiry_seconds' in group_df.columns:
        group_df['time_to_expiry_seconds'] = pd.to_numeric(
            group_df['time_to_expiry_seconds'],
            errors='coerce'
        ).astype(np.float32)
    group_df = group_df.dropna(subset=['target'] + covariate_cols)
    clean_groups[symbol] = group_df.reset_index(drop=True)

raw_target_series = {}
raw_covariate_series = {}

for symbol, group_df in clean_groups.items():
    target_series = TimeSeries.from_dataframe(
        group_df, 
        time_col='timestamp_fixed', 
        value_cols='target'
    ).astype(np.float32)
    
    covariate_series = TimeSeries.from_dataframe(
        group_df,
        time_col='timestamp_fixed',
        value_cols=covariate_cols
    ).astype(np.float32)
    
    raw_target_series[symbol] = target_series
    raw_covariate_series[symbol] = covariate_series


train_ratio = 0.8
train_target_series = []
test_target_series = []
train_covariate_series = []
test_covariate_series = []
symbols = list(raw_target_series.keys())
for symbol in symbols:
    target_series = raw_target_series[symbol]
    covariate_series = raw_covariate_series[symbol]
    
    split_point = int(len(target_series) * train_ratio)

    scaler_target = Scaler()
    scaler_covariate = Scaler()
    
    target_scaled = scaler_target.fit_transform(target_series).astype(np.float32)
    covariate_scaled = scaler_covariate.fit_transform(covariate_series).astype(np.float32)
    
    train_target_series.append(target_scaled[:split_point])
    test_target_series.append(target_scaled[split_point:])
    train_covariate_series.append(covariate_scaled[:split_point])
    test_covariate_series.append(covariate_scaled[split_point:])


rnn_model = RNNModel(
    model='LSTM',
    input_chunk_length=500,
    output_chunk_length=H,
    hidden_dim=128,
    n_rnn_layers=3,
    dropout=0.1,
    batch_size=32,
    n_epochs=1,
    random_state=42,
    training_length=500 + H,
    optimizer_kwargs={'lr': 1e-3},
    force_reset=True,
    pl_trainer_kwargs={
        'precision': '32-true',  
        'accelerator': 'auto'
    }
)

# Training
rnn_model.fit(
    series=train_target_series,
    future_covariates=train_covariate_series,
    verbose=True
)

mae_scores_per_symbol_rnn = {}
rmse_scores_per_symbol_rnn = {}
mape_scores_per_symbol_rnn = {}
predictions_per_symbol_rnn = {}

for i, symbol in enumerate(symbols):
    predicted = rnn_model.historical_forecasts(
        series=all_target_series_scaled[i],
        future_covariates=all_covariate_series_scaled[i], 
        start=len(train_target_series[i]),
        forecast_horizon=H,
        stride=1,
        retrain=False,
        verbose=False
    )
    predictions_per_symbol_rnn[symbol] = predicted

for i, symbol in enumerate(symbols):
    actual = target_scalers[i].inverse_transform(test_target_series[i])
    predicted = target_scalers[i].inverse_transform(predictions_per_symbol_rnn[symbol])

    mae_val = mae(actual, predicted)
    rmse_val = rmse(actual, predicted)
    try:
        mape_val = mape(actual, predicted)
    except ValueError as e:
        print(f"MAPE error for symbol {symbol}: {e}")
        mape_val = np.nan

    mae_scores_per_symbol_rnn[symbol] = mae_val
    rmse_scores_per_symbol_rnn[symbol] = rmse_val
    mape_scores_per_symbol_rnn[symbol] = mape_val

avg_mae_rnn = np.nanmean(list(mae_scores_per_symbol_rnn.values()))
avg_rmse_rnn = np.nanmean(list(rmse_scores_per_symbol_rnn.values()))
avg_mape_rnn = np.nanmean(list(mape_scores_per_symbol_rnn.values()))


metrics_df_rnn = pd.DataFrame({
    'Symbol': list(mae_scores_per_symbol_rnn.keys()),
    'MAE_RNN': list(mae_scores_per_symbol_rnn.values()),
    'RMSE_RNN': list(rmse_scores_per_symbol_rnn.values()),
    'MAPE_RNN': list(mape_scores_per_symbol_rnn.values())
})


print(metrics_df_rnn)

print("\n--- RNN Average Metrics ---")
print(f"RNN Average MAE: {avg_mae_rnn:.4f}")
print(f"RNN Average RMSE: {avg_rmse_rnn:.4f}")
print(f"RNN Average MAPE: {avg_mape_rnn:.4f}%")

metrics_df_rnn.to_pickle('metrics_one4all_10days_raw_RNN.pkl')

In [108]:
folder_path = "/gpfs/home5/talkanbar/"
file_name = "df_classified_good.pkl"
file_path = os.path.join(folder_path, file_name)
df.to_pickle(file_path)